# Import necessary Python libraries

In [ ]:
!pip install xgboost
!pip install segyio
!pip install segysak
!pip install seaborn
!pip install plotly
!pip install scikit-learn
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib
import xgboost as xgb
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score

# Data Import


In [ ]:
train = pd.read_csv('/data/CSV_train.csv', sep=';')

test = pd.read_csv('/data/CSV_test.csv', sep=';')

In [ ]:
train.sample(10)

# Exploratory Data Analysis & Visualization

## EDA

Let us look at the number of wells for training and testing in the dataset.

In [ ]:
Alldata=pd.concat([train, test], axis=0).reset_index().rename(columns={'index': 'idx'})
Alldata.loc[train.index, 'Dataset'] = 'Train'
Alldata.loc[train.index.max()+1:, 'Dataset'] = 'Test'

In [ ]:
train_wells = Alldata['WELL'][Alldata.Dataset=='Train'].unique()
print('No of train wells: %s' % len(train_wells))

In [ ]:
test_wells = Alldata['WELL'][Alldata.Dataset=='Test'].unique()
print('No of test wells: %s' % len(test_wells))

We will create a dictionary of the lithology names with respect to their unique number key present in the dataset.

In [ ]:
lithology_keys = {} # initialize dictionary
litho=['Sandstone', 'Shale', 'SandyShale', 'Limestone', 'Chalk', 'Dolomite',
       'Marl', 'Anhydrite', 'Halite', 'Coal', 'Basement', 'Tuff']
col = np.sort(Alldata['FORCE_2020_LITHOFACIES_LITHOLOGY'][Alldata.Dataset=='Train'].unique())
for index, name in enumerate(col):
    lithology_keys[name] = litho[index]
lithology_keys = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}
lithology_keys

Let us look at the count of different facies labels. 

In [ ]:
counts = train['FORCE_2020_LITHOFACIES_LITHOLOGY'].value_counts()
names = []
percentage = []
N = train['FORCE_2020_LITHOFACIES_LITHOLOGY'].shape[0]
for item in counts.items():
    names.append(lithology_keys[item[0]])
    percentage.append(float(item[1])/N*100)
fig, ax = plt.subplots(1, 1, figsize=(14, 7))
ax.bar(x=np.arange(len(names)), height=percentage)
ax.set_xticklabels(names, rotation=45)
ax.set_xticks(np.arange(len(names)))
ax.set_ylabel('Lithology presence (\%)')

Create a list of feature names to be used for training the ML model. 

In [ ]:
feature_names = ['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'GR', 'RSHA', 'RMED', 'RDEP', 'RHOB',
                 'NPHI', 'PEF', 'DTC', 'DTS', 'SP', 'ROP', 'BS']

We will create a new dictionary of sequential facies numbering with respect to the unique lithology numbers in the dataset.  

In [ ]:
Y = Alldata['FORCE_2020_LITHOFACIES_LITHOLOGY']
lithology_numbers = {} # initialize dictionary
lithology_numbers = {30000: 0,
                     65030: 1,
                     65000: 2,
                     80000: 3,
                     74000: 4,
                     70000: 5,
                     70032: 6,
                     88000: 7,
                     86000: 8,
                     99000: 9,
                     90000: 10,
                     93000: 11}
display(lithology_keys)
Y = Y.map(lithology_numbers)
Y.unique()

Store well labels and depths for training data

In [ ]:
well = Alldata['WELL']
dataset = Alldata[['idx', 'Dataset']]
depth = Alldata['DEPTH_MD']
Strat = Alldata['GROUP']
Formation = Alldata['FORMATION']

In [ ]:
X_all = Alldata.drop(columns=['WELL', 'FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE',
                              'idx', 'Dataset'])
X=X_all[feature_names]

Y=Y.reindex(X.index)
Strat=Strat.reindex(X.index)
Formation=Formation.reindex(X.index)

Fill null values with 0.

In [ ]:
X_imp=X.fillna(0)
X_imp.describe()

## Log plot

Create a dictionary of unique colours for each facies label, for visualization purposes.

In [ ]:
facies_names = lithology_keys.values()
facies_colors = ['darkorange', '#228B22', 'grey', 'cyan', 'gold', 'lightseagreen',
                 'lawngreen', 'lightblue', 'tan', '#FF4500', '#000000', 'magenta']
#
#facies_color_map is a dictionary that maps facies labels to their respective colors
facies_color_map = {}
for ind, label in enumerate(facies_names):
    facies_color_map[label] = facies_colors[ind]

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
def make_facies_log_plot(logs, curves, well_name, facies_colors):

    #make sure logs are sorted by depth
    cmap_facies = colors.ListedColormap(facies_colors, 'indexed')

    colours=['k','b','r','g','m','c','lime','gold','sienna']

    ztop=logs.Depth.min(); zbot=logs.Depth.max()

    cluster=np.repeat(logs['litho_real'].values.reshape(-1, 1),50,axis=1)

    num_curves = len(curves)
    f, ax = plt.subplots(nrows=1, ncols=num_curves+1, figsize=(num_curves*2, 12))

    for ic, col in enumerate(curves):

        # if the curve doesn't exist, make it zeros
        if np.all(np.isnan(logs[col])):
            curve = np.empty(logs[col].values.shape)
            curve[:] = np.nan

        else:
            curve = logs[col]

        ax[ic].plot(curve, logs.Depth,colours[ic])
        ax[ic].set_xlabel(col)
        if ic != 0:
            ax[ic].set_yticklabels([]);

    # make the lithfacies column
    im=ax[num_curves].imshow(cluster, interpolation='none', aspect='auto',
                    cmap=cmap_facies,vmin=0,vmax=len(facies_colors)-1)

    divider = make_axes_locatable(ax[num_curves])
    cax = divider.append_axes("right", size="20%", pad=0.05)
    cbar=plt.colorbar(im, cax=cax)
    cbar.set_label((13*' ').join(['  SS', 'SS-Sh', 'Sh',
                                ' Marl', 'Dol', 'Lims', 'Chlk ',
                                '  Hal', 'Anhy', 'Tuf', 'Coal', 'Bsmt']))
    cbar.set_ticks(range(0,1)); cbar.set_ticklabels('')

    for i in range(len(ax)-1):
        ax[i].set_ylim(ztop,zbot)
        ax[i].invert_yaxis()
        ax[i].grid()
        ax[i].locator_params(axis='x', nbins=3)

    ax[0].set_ylabel("DEPTH")
    ax[num_curves].set_xlabel('Lithology')
    ax[num_curves].set_yticklabels([])
    ax[num_curves].set_xticklabels([])

    f.suptitle('Well: %s'% well_name, fontsize=14,y=0.94)

In [ ]:
well_no=25
logs = pd.concat([X_imp,
                  Y], axis=1).rename(columns = {'FORCE_2020_LITHOFACIES_LITHOLOGY':'litho_real', 'DEPTH_MD':'Depth'
                                               })
make_facies_log_plot(logs[well==train_wells[well_no]], ['CALI', 'GR', 'RHOB', 'NPHI', 'RSHA', 'RDEP', 'PEF'],
                     train_wells[well_no],
                     facies_colors)

# Preprocessing and Model Training

## Preprocessing functions

In [ ]:
def drop_columns(data, *args):

    '''
    function used to drop columns.
    args::
      data:  dataframe to be operated on
      *args: a list of columns to be dropped from the dataframe

    return: returns a dataframe with the columns dropped
    '''

    columns = []
    for _ in args:
        columns.append(_)

    data = data.drop(columns, axis=1)

    return data


In [ ]:
def process(data):

    '''
    function to process dataframe by replacing missing, infinity values with -999

    args::
      data:  dataframe to be operated on

    returns dataframe with replaced values
    '''

    cols = list(data.columns)
    for _ in cols:

        data[_] = np.where(data[_] == np.inf, -999, data[_])
        data[_] = np.where(data[_] == np.nan, -999, data[_])
        data[_] = np.where(data[_] == -np.inf, -999, data[_])

    return data

## Feature Engineering

In [ ]:
# Feature windows concatenation function
def augment_features_window(X, N_neig):

    # Parameters
    N_row = X.shape[0]
    N_feat = X.shape[1]

    # Zero padding
    X = np.vstack((np.zeros((N_neig, N_feat)), X, (np.zeros((N_neig, N_feat)))))

    # Loop over windows
    X_aug = np.zeros((N_row, N_feat*(2*N_neig+1)))
    for r in np.arange(N_row)+N_neig:
        this_row = []
        for c in np.arange(-N_neig,N_neig+1):
            this_row = np.hstack((this_row, X[r+c]))
        X_aug[r-N_neig] = this_row

    return X_aug

In [ ]:
# Feature gradient computation function
def augment_features_gradient(X, depth):

    # Compute features gradient
    d_diff = np.diff(depth).reshape((-1, 1))
    d_diff[d_diff==0] = 0.001
    X_diff = np.diff(X, axis=0)
    X_grad = X_diff / d_diff

    # Compensate for last missing value
    X_grad = np.concatenate((X_grad, np.zeros((1, X_grad.shape[1]))))

    return X_grad

In [ ]:
# Combined feature augmentation function
def augment_features(X, well, depth, N_neig=1):

    # Augment features
    X_aug = np.zeros((X.shape[0], X.shape[1]*(N_neig*2+2)))
    for w in np.unique(well):
        w_idx = np.where(well == w)[0]
        X_aug_win = augment_features_window(X[w_idx, :], N_neig)
        X_aug_grad = augment_features_gradient(X[w_idx, :], depth[w_idx])
        X_aug[w_idx, :] = np.concatenate((X_aug_win, X_aug_grad), axis=1)

    # Find padded rows
    padded_rows = np.unique(np.where(X_aug[:, 0:7] == np.zeros((1, 7)))[0])

    return X_aug, padded_rows

# Model Architecture

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def show_evaluation(predictions, true_labels):
    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    print(f'Accuracy is: {accuracy}')

    # Generate and print classification report
    class_report = classification_report(true_labels, predictions)
    print(f'Classification Report:\n{class_report}')

    return accuracy

In [ ]:
class Model():

    '''
    class to lithology prediction
    '''

    def __init__(self, train, test):

        '''
        takes in the train and test dataframes
        '''

        self.train = train
        self.test = test


    def __call__(self, plot = False):

      return self.fit(plot)

    def preprocess(self, train, test):

        '''
        method to prepare datasets for training and predictions
        accepts both the train and test dataframes as arguments

        returns the prepared train, test datasets along with the
        lithology labels and numbers.

        '''

        #concatenating both train and test datasets for easier and uniform processing

        ntrain = train.shape[0]
        ntest = test.shape[0]
        target = train.FORCE_2020_LITHOFACIES_LITHOLOGY.copy()
        df = pd.concat((train, test)).reset_index(drop=True)

        #mapping the lithology labels to ordinal values for better modelling

        lithology = train['FORCE_2020_LITHOFACIES_LITHOLOGY']

        lithology_numbers = {30000: 0,
                        65030: 1,
                        65000: 2,
                        80000: 3,
                        74000: 4,
                        70000: 5,
                        70032: 6,
                        88000: 7,
                        86000: 8,
                        99000: 9,
                        90000: 10,
                        93000: 11}

        lithology1 = lithology.map(lithology_numbers)

        #implementing Bestagini's augmentation procedure

        train_well = train.WELL.values
        train_depth = train.DEPTH_MD.values

        test_well = test.WELL.values
        test_depth = test.DEPTH_MD.values



        print(f'shape of concatenated dataframe before dropping columns {df.shape}')

        cols = ['FORCE_2020_LITHOFACIES_CONFIDENCE', 'SGR', 'DTS', 'RXO', 'ROPA'] #columns to be dropped
        df = drop_columns(df, *cols)
        print(f'shape of dataframe after dropping columns {df.shape}')
        print(f'{cols} were dropped')

        #Label encoding the GROUP, FORMATION and WELLS features as these improved the performance of the models on validations

        df['GROUP_encoded'] = df['GROUP'].astype('category')
        df['GROUP_encoded'] = df['GROUP_encoded'].cat.codes
        df['FORMATION_encoded'] = df['FORMATION'].astype('category')
        df['FORMATION_encoded'] = df['FORMATION_encoded'].cat.codes
        df['WELL_encoded'] = df['WELL'].astype('category')
        df['WELL_encoded'] = df['WELL_encoded'].cat.codes
        print(f'shape of dataframe after label encoding columns {df.shape}')


        #FURTHER PREPRATION TO SPLIT DATAFRAME INTO TRAIN AND TEST DATASETS AFTER PREPRATION
        print(f'Splitting concatenated dataframe into training and test datasets...')
        df = df.drop(['WELL', 'GROUP', 'FORMATION'], axis=1)
        print(df.shape)

        df = df.fillna(-999)
        df = process(df)
        data = df.copy()

        train2 = data[:ntrain].copy()
        train2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)

        test2 = data[ntrain:(ntest+ntrain)].copy()
        test2.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY'], axis=1, inplace=True)
        test2 = test2.reset_index(drop=True)

        traindata = train2
        testdata = test2

        print(f'Shape of train and test datasets before augmentation {traindata.shape, testdata.shape}')

        traindata1, padded_rows = augment_features(pd.DataFrame(traindata).values, train_well, train_depth)
        testdata1, padded_rows = augment_features(pd.DataFrame(testdata).values, test_well, test_depth)

        print(f'Shape of train and test datasets after augmentation {traindata1.shape, testdata1.shape}')

        return traindata1, testdata1, lithology1, lithology_numbers

    # ___________________________________________________________________________________________________________________________________________________________
    def fit(self, plot):

      '''
      method to train model and make predictions

      returns the test predictions, trained model, and lithology numbers used for making the submission file
      '''

      traindata1, testdata1, lithology1, lithology_numbers = self.preprocess(self.train, self.test)

      #using a 10-fold stratified cross-validation technique and seting the shuffle parameter to true
      #as this improved the validation performance better

      split = 10
      kf = StratifiedKFold(n_splits=split, shuffle=True)

      open_test = np.zeros((len(testdata1), 12))

      #100 n-estimators and 10 max-depth
      model = XGBClassifier(n_estimators=100, max_depth=10, booster='gbtree',
                            objective='multi:softprob', learning_rate=0.1, random_state=0,
                            subsample=0.9, colsample_bytree=0.9,
                            eval_metric='mlogloss', verbose=2020, reg_lambda=1500)


      i = 1
      for (train_index, test_index) in kf.split(pd.DataFrame(traindata1), pd.DataFrame(lithology1)):
        X_train, X_test = pd.DataFrame(traindata1).iloc[train_index], pd.DataFrame(traindata1).iloc[test_index]
        Y_train, Y_test = pd.DataFrame(lithology1).iloc[train_index],pd.DataFrame(lithology1).iloc[test_index]

        model.fit(X_train, Y_train, early_stopping_rounds=100, eval_set=[(X_test, Y_test)], verbose=100)
        prediction = model.predict(X_test)
        print(show_evaluation(prediction, Y_test))

        print(f'-----------------------FOLD {i}---------------------')
        i+=1

        open_test += model.predict_proba(pd.DataFrame(testdata1))

      open_test= pd.DataFrame(open_test/split)

      open_test = np.array(pd.DataFrame(open_test).idxmax(axis=1))

      print('---------------CROSS VALIDATION COMPLETE')
      print('----------------TEST EVALUATION------------------')


      return open_test, model, lithology_numbers



# Model Training

In [ ]:
func_= Model(train, test)
prediction, model, redundant = func_()

# Results

In [ ]:
selected_well = train[train['WELL'] == '15/9-17']

features = selected_well.drop(columns=['GROUP', 'FORMATION'])

processed_features = func_.preprocess(features, test)[0]

well_predictions = model.predict(processed_features)

well_predictions_labels = [[key for key, value in redundant.items() if value == label] for label in well_predictions]

# print("Predicted Lithology Labels for all instances in the well:", well_predictions_labels)

In [ ]:
original = selected_well['FORCE_2020_LITHOFACIES_LITHOLOGY']
series = pd.Series([item for sublist in well_predictions_labels for item in sublist])

import matplotlib.pyplot as plt
import numpy as np


facies_mapping = {
    30000: 'Sandstone',
    65030: 'Sandstone/Shale',
    65000: 'Shale',
    80000: 'Marl',
    74000: 'Dolomite',
    70000: 'Limestone',
    70032: 'Chalk',
    88000: 'Halite',
    86000: 'Anhydrite',
    99000: 'Tuff',
    90000: 'Coal',
    93000: 'Basement'
}

facies_colors = {
    'Sandstone': 'red',
    'Sandstone/Shale': 'blue',
    'Shale': 'green',
    'Marl': 'yellow',
    'Dolomite': 'purple',
    'Limestone': 'orange',
    'Chalk': 'pink',
    'Halite': 'brown',
    'Anhydrite': 'gray',
    'Tuff': 'cyan',
    'Coal': 'black',
    'Basement': 'magenta'
}


colors_original = [facies_colors[facies_mapping[facies]] for facies in original]
colors_series = [facies_colors[facies_mapping[facies]] for facies in series]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

scatter1 = ax1.scatter(range(len(original)), [1] * len(original), c=colors_original, marker='|', s=1000)
ax1.set_yticks([])
ax1.set_title('Original Data')
scatter2 = ax2.scatter(range(len(series)), [1] * len(series), c=colors_series, marker='|', s=1000)
ax2.set_xlabel('Data Point')
ax2.set_yticks([])
ax2.set_title('Prediction')
legend_labels = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in facies_colors.items()]
fig.legend(handles=legend_labels,  loc='center right',ncol=2)
plt.tight_layout()
plt.show()

